In [ ]:
import os
import cv2
import numpy as np
from PIL import Image, ImageEnhance

# Set your folder path
folder_path = r"C://Users//admin//Downloads//Milind Thesis//Dataset//Cross Roads"

# Supported image extensions
supported_exts = ['.jpg', '.jpeg', '.png', '.bmp']

# Load all image files
image_files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in supported_exts]

def save_image(img, base_name, suffix):
    filename = f"{os.path.splitext(base_name)[0]}_{suffix}.png"
    cv2.imwrite(os.path.join(folder_path, filename), img)

def apply_blur(img, base_name):
    for sigma in [3, 5, 7, 9]:
        ksize = (sigma * 2 + 1, sigma * 2 + 1)
        blurred = cv2.GaussianBlur(img, ksize, sigmaX=sigma)
        save_image(blurred, base_name, f"blur_sigma{sigma}")

def apply_haze(img, base_name):
    haze_layer = np.full_like(img, fill_value=(128, 128, 128), dtype=np.uint8)
    for opacity in [0.3, 0.5, 0.7, 0.9]:
        hazy = cv2.addWeighted(img, 1 - opacity, haze_layer, opacity, 0)
        save_image(hazy, base_name, f"haze_opacity{int(opacity*100)}")

def apply_low_light(img, base_name):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img_bright = ImageEnhance.Brightness(img_pil).enhance(0.7)
    img_contrast = ImageEnhance.Contrast(img_bright).enhance(0.7)
    for gamma in [0.3, 0.5, 0.7, 0.9]:
        lut = [((i / 255.0) ** gamma) * 255 for i in range(256)]
        lut = np.array(lut, dtype='uint8')
        gamma_corrected = cv2.LUT(np.array(img_contrast.convert('RGB')), lut)
        save_image(cv2.cvtColor(gamma_corrected, cv2.COLOR_RGB2BGR), base_name, f"lowlight_gamma{gamma}")

def apply_rotation(img, base_name):
    (h, w) = img.shape[:2]
    center = (w // 2, h // 2)
    for angle in [20, 50, 70, 90]:
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR)
        save_image(rotated, base_name, f"rot{angle}")

def apply_rain(img, base_name):
    (h, w) = img.shape[:2]
    for noise_level in [0.03, 0.05, 0.07, 0.09]:
        rain_layer = np.zeros((h, w), dtype=np.uint8)
        num_drops = int(noise_level * h * w)
        for _ in range(num_drops):
            x, y = np.random.randint(0, w), np.random.randint(0, h)
            cv2.line(rain_layer, (x, y), (x, y + 10), 255, 1)
        rain_layer = cv2.blur(rain_layer, (1, 3))
        rain_color = cv2.cvtColor(rain_layer, cv2.COLOR_GRAY2BGR)
        rain_img = cv2.addWeighted(img, 0.8, rain_color, 0.2, 0)
        save_image(rain_img, base_name, f"rain_noise{noise_level:.2f}")

def apply_shadow(img, base_name):
    h, w = img.shape[:2]
    for opacity in [0.03, 0.05, 0.07, 0.09]:
        shadow_mask = np.ones((h, w, 3), dtype=np.float32)
        shadow_mask[:, :w // 2] -= opacity
        shadowed = np.clip(img * shadow_mask, 0, 255).astype(np.uint8)
        save_image(shadowed, base_name, f"shadow_opacity{int(opacity*100)}")

# Apply all transformations
for file in image_files:
    img_path = os.path.join(folder_path, file)
    img = cv2.imread(img_path)

    apply_blur(img, file)
    apply_haze(img, file)
    apply_low_light(img, file)
    apply_rotation(img, file)
    apply_rain(img, file)
    apply_shadow(img, file)

print("All transformations applied and saved.")
